In [1]:
import requests
import torch
import numpy as np
import pandas as pd
from latentsae import Sae
from sentence_transformers import SentenceTransformer

Triton not installed, using eager implementation of SAE decoder.


In [2]:
# first load the model
sae_model = Sae.load_from_hub("enjalot/sae-nomic-text-v1.5-FineWeb-edu-100BT", "64_32", device="cpu")
chunk_mapping = requests.get("https://enjalot.github.io/latent-taxonomy/models/NOMIC_FWEDU_25k/chunk_mapping.json").json()
sae_meta = requests.get("https://enjalot.github.io/latent-taxonomy/models/NOMIC_FWEDU_25k/metadata.json").json()
sae_features = pd.read_parquet("https://enjalot.github.io/latent-taxonomy/models/NOMIC_FWEDU_25k/features.parquet")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Dropping extra args {'signed': False}


In [3]:
emb_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True, device="cpu")

<All keys matched successfully>


In [4]:
# get the top 10 samples for this feature
def get_samples(feature):
    chunk = chunk_mapping[str(feature)]
    samples = pd.read_parquet(f"https://enjalot.github.io/latent-taxonomy/models/NOMIC_FWEDU_25k/samples/chunk_{chunk}.parquet")
    samples = samples[samples["feature"] == feature]
    return samples

In [5]:
#feature = 4658
# feature = 11789
# feature = 719
feature = 0

In [6]:
samples = get_samples(feature)
samples.head()

,id,text,url,feature,activation,top_acts,top_indices
0,<urn:uuid:ade93a67-14e4-4835-8fb6-7668fe8004db>,- simple past tense and past participle of gel...,http://www.yourdictionary.com/gelled,0,0.259158,"[0.2783777713775635, 0.2591576874256134, 0.113...","[2006.0, 0.0, 793.0, 3718.0, 5826.0, 10367.0, ..."
1,<urn:uuid:ae2f5cf3-6392-4160-a4ee-5b36b3a97e65>,"Also found in: Thesaurus, Medical, Acronyms, E...",https://www.thefreedictionary.com/gelatin,0,0.256561,"[0.2565605938434601, 0.22474993765354156, 0.13...","[0.0, 2006.0, 5826.0, 793.0, 5945.0, 21665.0, ..."
2,<urn:uuid:c33df90e-c479-40e2-aa9b-ac6a4f0e47d9>,"A gel (from the lat. gelu—freezing, cold, ice ...",http://medicalxpress.com/tags/hydrogel/sort/li...,0,0.245421,"[0.24542075395584106, 0.09371058642864227, 0.0...","[0.0, 2006.0, 10367.0, 7655.0, 16380.0, 15780...."
3,<urn:uuid:16add753-dbb9-44e2-aad0-b9909c16cff5>,extended periods of time can damage gelatin an...,http://mlaiskonis.com/2014/06/07/gelatin/,0,0.243735,"[0.24373508989810944, 0.15119953453540802, 0.1...","[0.0, 5826.0, 21665.0, 12673.0, 16225.0, 4502...."
4,<urn:uuid:acd089d5-9546-4c25-8142-ab2a5e3204cc>,##loid gel. gelatin forms a solution of high v...,http://www.foodfacts.com/ci/ingredientsoverlay...,0,0.242724,"[0.2427240014076233, 0.1294490396976471, 0.087...","[0.0, 5826.0, 13791.0, 7655.0, 11963.0, 15780...."


In [7]:
all_samples = pd.read_parquet("~/code/latent-taxonomy/notebooks/data/top10_64_32.parquet")

In [8]:
asamples = all_samples[all_samples["feature"] == feature]

In [9]:
asamples.head()

,chunk_index,chunk_text,chunk_token_count,id,url,score,dump,embedding,top_acts,top_indices,feature,activation
0,0,"2019 Study Abstract\nGenomic imprinting, the m...",191,<urn:uuid:d45d32f3-aee4-464b-a7a7-4659ca6f95a5>,https://desdaughter.com/2019/01/21/genomic-imp...,2.859375,CC-MAIN-2019-18,"[0.070664756, 0.04058804, -0.1678945, -0.04720...","[0.29729074239730835, 0.23520702123641968, 0.1...","[0.0, 19961.0, 19487.0, 3596.0, 9132.0, 16563....",0,0.297291
4973,1,biological function and regulation of imprinte...,500,<urn:uuid:6492a5df-795c-4afc-8b96-adda43d374fe>,http://www.biomedcentral.com/1471-2164/10/144,2.640625,CC-MAIN-2015-11,"[0.08591175, 0.05282476, -0.1623837, -0.038760...","[0.2801821529865265, 0.21019425988197327, 0.11...","[0.0, 19961.0, 12474.0, 18618.0, 5676.0, 18178...",0,0.280182
2426,0,Molecular imprinting is a technique used to cr...,277,<urn:uuid:c4cec9f7-a221-4bac-8872-ad938bbe3b9c>,https://www.advancedsciencenews.com/new-trends...,2.796875,CC-MAIN-2021-04,"[0.06389447, 0.054419804, -0.1866366, -0.05261...","[0.27958399057388306, 0.15807494521141052, 0.1...","[0.0, 21919.0, 18672.0, 3614.0, 13226.0, 15727...",0,0.279584
44507,0,[CLS] imprinting / do not go where the path ma...,500,<urn:uuid:fd9748b7-ad11-4d51-b7d9-b5681c579e36>,https://www.windermeresun.com/2017/08/05/impri...,3.218750,CC-MAIN-2023-40,"[0.045871254, 0.0841982, -0.20583852, -0.07991...","[0.27171218395233154, 0.24269740283489227, 0.1...","[0.0, 6864.0, 8104.0, 3020.0, 15020.0, 8543.0,...",0,0.271712
7496,0,There have been a number of recent insights in...,212,<urn:uuid:40c30498-bed6-4b01-a37e-a2a1b70d80fd>,https://pure.ulster.ac.uk/en/publications/impr...,3.078125,CC-MAIN-2021-10,"[0.06898261, 0.04841869, -0.16367012, -0.05253...","[0.2717033922672272, 0.2493799477815628, 0.117...","[0.0, 19961.0, 21919.0, 14900.0, 22498.0, 1514...",0,0.271703


In [10]:
# embedding = emb_model.encode(samples["text"].to_list())
embedding = emb_model.encode(asamples["chunk_text"].to_list())
embedding.shape

(10, 768)

In [11]:
normalized_embeddings = embedding / np.linalg.norm(embedding, ord=2, axis=1, keepdims=True)

In [12]:
aembeddings = asamples["embedding"].to_numpy()
aembeddings = np.array([np.array(emb) for emb in aembeddings])

In [13]:
# extract the topk features (directions) for each embedding
# directions = sae_model.encode(torch.from_numpy(embedding))
# directions = sae_model.encode(torch.from_numpy(normalized_embeddings))
directions = sae_model.encode(torch.from_numpy(aembeddings))

In [14]:
print(directions.top_acts.shape)
print(directions.top_indices.shape)

torch.Size([10, 64])
torch.Size([10, 64])


In [15]:
# visualize topk for a single sample
directions.top_indices[0]

tensor([    0, 19961, 19487,  3596,  9132, 16563, 18672, 15146, 22498, 18178,
         1893,   928, 15020,  2305,  6674, 18703, 16053, 17574,  6869, 21919,
         2597, 22048, 21556, 17923, 17837, 13226, 17469,  4501, 10314,    38,
         1299,   304, 14900, 12474,  1309, 17717, 20007, 17054, 19394, 18618,
        10659, 24116,  2310, 17529,  2463, 10237,  7722, 10225,  7652, 20384,
        12970,  1883,  3185,  5676, 24487, 15884, 23890,  5174,  1450,  1051,
         3874, 23637, 13718,   363])

In [16]:
directions.top_acts[0]

tensor([0.2973, 0.2352, 0.1373, 0.0896, 0.0880, 0.0878, 0.0854, 0.0753, 0.0728,
        0.0666, 0.0570, 0.0548, 0.0522, 0.0514, 0.0432, 0.0431, 0.0417, 0.0413,
        0.0400, 0.0359, 0.0355, 0.0341, 0.0337, 0.0334, 0.0320, 0.0317, 0.0299,
        0.0295, 0.0283, 0.0272, 0.0268, 0.0256, 0.0254, 0.0251, 0.0250, 0.0245,
        0.0242, 0.0239, 0.0238, 0.0238, 0.0233, 0.0230, 0.0229, 0.0223, 0.0222,
        0.0222, 0.0221, 0.0219, 0.0218, 0.0217, 0.0217, 0.0213, 0.0208, 0.0207,
        0.0205, 0.0204, 0.0204, 0.0202, 0.0202, 0.0194, 0.0194, 0.0192, 0.0191,
        0.0188], grad_fn=<SelectBackward0>)

In [186]:
def get_direction(directions, idx):
    return { "top_indices": directions.top_indices[idx], "top_acts": directions.top_acts[idx] }


In [17]:
def get_maxk(directions, k=32):
    # Flatten the top_acts and top_indices to find the global top k
    flat_acts = directions.top_acts.flatten()
    flat_indices = directions.top_indices.flatten()

    # Create a dictionary to keep track of the highest activation for each index
    max_acts = {}
    
    for act, idx in zip(flat_acts, flat_indices):
        if idx.item() not in max_acts or act.item() > max_acts[idx.item()]:
            max_acts[idx.item()] = act.item()
    
    # Convert the dictionary back to tensors
    flat_acts = torch.tensor(list(max_acts.values()))
    flat_indices = torch.tensor(list(max_acts.keys()))
    
    # Get the top k activations and their corresponding indices
    topk_acts, topk_indices = torch.topk(flat_acts, k)
    
    # Map the flat indices back to the original indices
    original_indices = flat_indices[topk_indices]
    
    topk = {
        "top_acts": topk_acts,
        "top_indices": original_indices
    }
    return topk

In [18]:
maxk = get_maxk(directions, 32)

In [126]:
maxk

{'top_acts': tensor([0.2973, 0.2494, 0.2427, 0.1785, 0.1581, 0.1403, 0.1403, 0.1388, 0.1373,
         0.1342, 0.1219, 0.1175, 0.1096, 0.1033, 0.1014, 0.0998, 0.0983, 0.0899,
         0.0895, 0.0885, 0.0880, 0.0878, 0.0861, 0.0831, 0.0798, 0.0778, 0.0760,
         0.0753, 0.0750, 0.0723, 0.0712, 0.0710]),
 'top_indices': tensor([    0, 19961,  6864, 13226, 21919, 18672, 12474, 18178, 19487, 15020,
          2463,  8104,  5676, 18618, 14900, 12064,  3614,  3596, 12363,   668,
          9132, 16563, 22498, 18703, 17432, 20384,  3020, 15146, 15727,  1141,
          6726, 18874])}

In [182]:
from latentsae import TopkVisWidget

In [190]:
features = sae_features.to_dict(orient='records')

In [195]:
widget = TopkVisWidget(data=maxk, n=10, features=features)
widget

TopkVisWidget(data={'top_acts': [0.29729074239730835, 0.24937991797924042, 0.24269738793373108, 0.178472474217…

In [196]:
widget = TopkVisWidget(data=get_direction(directions, 0), n=10, features=features)
widget



TopkVisWidget(data={'top_indices': [0, 19961, 19487, 3596, 9132, 16563, 18672, 15146, 22498, 18178, 1893, 928,…